In [2]:
import sys
!{sys.executable} -m pip install -r requirements.txt
import pydicom
import matplotlib.pylab as plt
import matplotlib
import os
import numpy as np
from PIL import Image
import time


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


Configure Global Variables
----

In [3]:
window = 350 # Window/Level from Hoori et al ("HU-attention-window with a window/level of 350/40 HU")
level = 40
bitDepth = 8 # Must be 8 for uint8 PNGs. Explore floating point
TrainingDataRootDir = '../Training Data'
TestDataRootDir = '../Test Data'

Preprocess Training Data
----

In [5]:
CombinedTrainingImageDir = TrainingDataRootDir + '/AllImages'
CombinedTrainingMaskDir = TrainingDataRootDir + '/AllMasks'
os.makedirs(CombinedTrainingImageDir, exist_ok=True)
os.makedirs(CombinedTrainingMaskDir, exist_ok=True)

currentImageIndex = 0
subjects = os.listdir(TrainingDataRootDir)
for subject in subjects:
    if not (subject == 'AllImages' or subject == 'AllMasks' or subject == 'AllOutputImages' or subject == 'AllOutputMasks' or subject == 'AllReferenceImages' or subject == 'AllCombinedResultImages'):
        currentSubjectDir = TrainingDataRootDir + "/" + subject
        print(currentSubjectDir)

        volume_dicoms = os.listdir(currentSubjectDir + '/Dicom/Vol/')
        dataset_vol = [None] * (len(volume_dicoms))
        for dicom in volume_dicoms:
            slice_number = int(dicom.split('Slice')[1].split('.dcm')[0])
            dataset_vol[slice_number-1] = pydicom.dcmread(currentSubjectDir+'/Dicom/Vol/'+dicom)
        
        sac_dicoms = os.listdir(currentSubjectDir + '/Dicom/Sac/')
        dataset_sac = [None] * (len(sac_dicoms))
        for dicom in sac_dicoms:
            slice_number = int(dicom.split('Slice')[1].split('.dcm')[0])
            dataset_sac[slice_number-1] = pydicom.dcmread(currentSubjectDir+'/Dicom/Sac/'+dicom)

        for i in range(1, len(dataset_vol)-1):
            sliceRGB = np.zeros((dataset_vol[i].pixel_array.shape[0],dataset_vol[i].pixel_array.shape[1],3))
            sliceRGB[:,:,0] = dataset_vol[i-1].pixel_array
            sliceRGB[:,:,1] = dataset_vol[i].pixel_array
            sliceRGB[:,:,2] = dataset_vol[i+1].pixel_array
            sliceRGB = np.clip((sliceRGB - (level-window/2)) / window, 0,1) # Apply window/level and clip to 0->1 range
            sliceRGB = sliceRGB * ((2**bitDepth)-1) # Scale to bitdepth
            sliceRGB = np.floor(sliceRGB) # Floor to integer
            imageFilename = CombinedTrainingImageDir + "/" + str(currentImageIndex)+'.png'
            matplotlib.image.imsave(imageFilename, sliceRGB.astype(np.uint8))

            sliceMask = np.zeros((dataset_vol[i].pixel_array.shape[0],dataset_vol[i].pixel_array.shape[1],3))
            sliceMask[:,:,0] = dataset_sac[i].pixel_array
            sliceMask[:,:,1] = dataset_sac[i].pixel_array
            sliceMask[:,:,2] = dataset_sac[i].pixel_array
            sliceMask = sliceMask * ((2**bitDepth)-1) # Scale to bitdepth
            maskFilename = CombinedTrainingMaskDir + "/" + str(currentImageIndex)+'.png'
            matplotlib.image.imsave(maskFilename, sliceMask.astype(np.uint8))

            currentImageIndex = currentImageIndex + 1

../Training Data/SA209
../Training Data/CA003
../Training Data/CA011
../Training Data/CA055
../Training Data/SA218
../Training Data/SA221
../Training Data/CA044
../Training Data/CA002
../Training Data/CA013
../Training Data/SA204
../Training Data/SA206
../Training Data/CA054
../Training Data/CA064
../Training Data/CA019R
../Training Data/SA212
../Training Data/SA201
../Training Data/CA024
../Training Data/SA219
../Training Data/SA200
../Training Data/SA222
../Training Data/CA059
../Training Data/CA006
../Training Data/SA215
../Training Data/CA050
../Training Data/CA060
../Training Data/CA041
../Training Data/CA039
../Training Data/CA067
../Training Data/CA068
../Training Data/SA211
../Training Data/CA036
../Training Data/CA014
../Training Data/CA061
../Training Data/SA210
../Training Data/CA020
../Training Data/CA004
../Training Data/SA205
../Training Data/CA042
../Training Data/SA213
../Training Data/CA053


Preprocess Test Data
----

In [6]:
CombinedTestImageDir = TestDataRootDir + '/AllImages'
CombinedTestMaskDir = TestDataRootDir + '/AllMasks'
os.makedirs(CombinedTestImageDir, exist_ok=True)
os.makedirs(CombinedTestMaskDir, exist_ok=True)

currentImageIndex = 0
subjects = os.listdir(TestDataRootDir)
for subject in subjects:
    if not (subject == 'AllImages' or subject == 'AllMasks' or subject == 'AllOutputImages' or subject == 'AllOutputMasks' or subject == 'AllReferenceImages' or subject == 'AllCombinedResultImages'):
        currentSubjectDir = TestDataRootDir + "/" + subject;
        print(currentSubjectDir)

        volume_dicoms = os.listdir(currentSubjectDir + '/Dicom/Vol/')
        dataset_vol = [None] * (len(volume_dicoms))
        for dicom in volume_dicoms:
            slice_number = int(dicom.split('Slice')[1].split('.dcm')[0])
            dataset_vol[slice_number-1] = pydicom.dcmread(currentSubjectDir+'/Dicom/Vol/'+dicom)
        
        sac_dicoms = os.listdir(currentSubjectDir + '/Dicom/Sac/')
        dataset_sac = [None] * (len(sac_dicoms))
        for dicom in sac_dicoms:
            slice_number = int(dicom.split('Slice')[1].split('.dcm')[0])
            dataset_sac[slice_number-1] = pydicom.dcmread(currentSubjectDir+'/Dicom/Sac/'+dicom)

        for i in range(1, len(dataset_vol)-1):
            sliceRGB = np.zeros((dataset_vol[i].pixel_array.shape[0],dataset_vol[i].pixel_array.shape[1],3))
            sliceRGB[:,:,0] = dataset_vol[i-1].pixel_array
            sliceRGB[:,:,1] = dataset_vol[i].pixel_array
            sliceRGB[:,:,2] = dataset_vol[i+1].pixel_array
            sliceRGB = np.clip((sliceRGB - (level-window/2)) / window, 0,1) # Apply window/level and clip to 0->1 range
            sliceRGB = sliceRGB * ((2**bitDepth)-1) # Scale to bitdepth
            sliceRGB = np.floor(sliceRGB) # Floor to integer
            imageFilename = CombinedTestImageDir + "/" + str(currentImageIndex)+'.png'
            matplotlib.image.imsave(imageFilename, sliceRGB.astype(np.uint8))

            sliceMask = np.zeros((dataset_vol[i].pixel_array.shape[0],dataset_vol[i].pixel_array.shape[1],3))
            sliceMask[:,:,0] = dataset_sac[i].pixel_array
            sliceMask[:,:,1] = dataset_sac[i].pixel_array
            sliceMask[:,:,2] = dataset_sac[i].pixel_array
            sliceMask = sliceMask * ((2**bitDepth)-1) # Scale to bitdepth
            maskFilename = CombinedTestMaskDir + "/" + str(currentImageIndex)+'.png'
            matplotlib.image.imsave(maskFilename, sliceMask.astype(np.uint8))

            currentImageIndex = currentImageIndex + 1

../Test Data/SA201
../Test Data/CA024
../Test Data/CA039
../Test Data/CA036
../Test Data/CA061
../Test Data/CA020


Find Voxel Size
----

In [3]:
subjects = os.listdir(TestDataRootDir)
for subject in subjects:
    if not (subject == 'AllImages' or subject == 'AllMasks' or subject == 'AllOutputImages' or subject == 'AllOutputMasks' or subject == 'AllReferenceImages' or subject == 'AllCombinedResultImages'):
        currentSubjectDir = TestDataRootDir + "/" + subject
        print(currentSubjectDir)

        volume_dicoms = os.listdir(currentSubjectDir + '/Dicom/Vol/')
        dataset_vol = [None] * (len(volume_dicoms))
        for dicom in volume_dicoms:
            slice_number = int(dicom.split('Slice')[1].split('.dcm')[0])
            dataset_vol[slice_number-1] = pydicom.dcmread(currentSubjectDir+'/Dicom/Vol/'+dicom)
            print(dataset_vol[slice_number-1])




../Test Data/SA218
Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 230
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.9590.100.1.2.29992222513963930435459680611559470107
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.9590.100.1.3.100.9.4
(0002, 0013) Implementation Version Name         SH: 'MATLAB IPT 9.4'
(0002, 0016) Source Application Entity Title     AE: 'GDCM/ITK 5.1.1'
-------------------------------------------------
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.9590.100.1.2.299922225139639304354596806115594

In [ ]:
## CA
## (0018, 0050) Slice Thickness                     DS: '2.5'
## (0028, 0010) Rows                                US: 512
## (0028, 0011) Columns                             US: 512
## (0028, 0030) Pixel Spacing                       DS: [0.8671875, 0.8671875]


## SA
## (0018, 0050) Slice Thickness                     DS: '2.5'
## (0028, 0010) Rows                                US: 512
## (0028, 0011) Columns                             US: 512
## (0028, 0030) Pixel Spacing                       DS: [0.78125, 0.78125]
